##**FINE-TUNING BERT WITH MLM**

In [1]:
!nvidia-smi

Wed Jul 14 11:30:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

####**Install the huggingface transformers library**

In [ ]:
!pip install transformers

In [2]:
import torch
from tqdm.auto import tqdm
from transformers import AdamW
from transformers import BertTokenizer, BertForMaskedLM

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


####**Import text data**

In [4]:
data_path = '/content/the_fire_flower.txt'
with open(data_path, 'r') as f:
    data = f.read().split('\n')

####**Text cleaning process**

In [5]:
print(len(data))

3949


In [6]:
for sentence in data:
    if len(sentence) < 50:
        data.remove(sentence)

In [7]:
print(len(data))

3078


####**Tokenizing the text data**

In [8]:
inputs = tokenizer(
    data,
    max_length=512,
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)

In [9]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [10]:
inputs['labels'] = inputs['input_ids'].detach().clone()
inputs

{'input_ids': tensor([[  101,  1996,  2622,  ...,     0,     0,     0],
        [  101,  2023, 26885,  ...,     0,     0,     0],
        [  101,  2087,  2060,  ...,     0,     0,     0],
        ...,
        [  101,  8756,  3192,  ...,     0,     0,     0],
        [  101,  4942, 29234,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]]), 'labels': tensor([[  101,  1996,  2622,  ...,     0,     0,     0],
        [  101,  2023, 26885,  ...,     0,     0,     0],
        [  101,  2087, 

####**Masking the input_ids**

In [11]:
random_tensor = torch.rand(inputs['input_ids'].shape)

In [12]:
random_tensor.shape

torch.Size([3078, 512])

In [13]:
# creating a random tensor of float values.
random_tensor

tensor([[0.6974, 0.2318, 0.5568,  ..., 0.6542, 0.9087, 0.5681],
        [0.3298, 0.9107, 0.3362,  ..., 0.5691, 0.3070, 0.0999],
        [0.7851, 0.3420, 0.4899,  ..., 0.9055, 0.7023, 0.8014],
        ...,
        [0.8422, 0.2042, 0.4212,  ..., 0.2519, 0.5127, 0.1659],
        [0.7554, 0.4178, 0.4344,  ..., 0.4360, 0.1397, 0.8117],
        [0.8325, 0.0738, 0.8135,  ..., 0.4894, 0.0600, 0.1216]])

In [14]:
# creating a mask tensor of float values ranging from 0 to 1 and avoiding special tokens
masked_tensor = (random_tensor < 0.15)*(inputs['input_ids'] != 101)*(inputs['input_ids'] != 102)*(inputs['input_ids'] != 0)

In [15]:
# getting all those indices from each row which are set to True, i.e. masked.
nonzeros_indices = []
for i in range(len(masked_tensor)):
    nonzeros_indices.append(torch.flatten(masked_tensor[i].nonzero()).tolist())

In [16]:
# setting the values at those indices to be a MASK token (103) for every row in the original input_ids.
for i in range(len(inputs['input_ids'])):
    inputs['input_ids'][i, nonzeros_indices[i]] = 103

####**Pytorch Dataset and Dataloader**

In [17]:
class BookDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, index):
        input_ids = self.encodings['input_ids'][index]
        labels = self.encodings['labels'][index]
        attention_mask = self.encodings['attention_mask'][index]
        token_type_ids = self.encodings['token_type_ids'][index]
        return {
            'input_ids': input_ids,
            'labels': labels,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids
        }

In [18]:
dataset = BookDataset(inputs)

In [19]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=16,
    shuffle=True
)

In [20]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [ ]:
model.to(device)

####**Model parameters**

In [22]:
epochs = 2
optimizer = AdamW(model.parameters(), lr=1e-5)

####**Training Loop**

In [ ]:
model.train()

for epoch in range(epochs):
    loop = tqdm(dataloader)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backwards()
        optimizer.step()

        loop.set_description("Epoch: {}".format(epoch))
        loop.set_postfix(loss=loss.item())

  0%|          | 0/193 [00:00<?, ?it/s]